In [7]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely import Polygon

In [8]:
regions = gpd.read_parquet('/data/uscuni-ulce/regions/cadastre_regions_hull.parquet')


In [11]:
# !unzip PMO\ ITI.zip

In [12]:
bounds = gpd.read_file('PMO ITI/').to_crs(epsg=3035)
bounds = bounds.union_all().exterior
bounds = Polygon(bounds.coords)

In [17]:
# gpd.GeoSeries(bounds, crs=3035).explore()

In [23]:
bounds_regions = regions[regions.intersects(bounds)].index.values

In [24]:
%%time

bounds_etcs = []

for rid in bounds_regions:
    print(rid)
    region_buildings = gpd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/buildings_chars_{rid}.parquet')
    region_buildings_within_bounds = region_buildings.index[region_buildings.sindex.query(bounds, predicate='intersects')].values
    
    region_chars = pd.read_parquet(f'/data/uscuni-ulce/processed_data/chars/primary_chars_{rid}.parquet')
    region_chars = region_chars.loc[region_buildings_within_bounds]

    to_save = gpd.GeoDataFrame(region_chars, 
                 geometry=region_buildings.loc[region_buildings_within_bounds].geometry,
                 crs=region_buildings.crs)

    to_save.index = str(rid) + '_' + to_save.index.astype(str)
    
    bounds_etcs.append(to_save)

67279
67803
69333
70973
73611
73735
75642
78792
CPU times: user 3.1 s, sys: 1.09 s, total: 4.18 s
Wall time: 2.04 s


In [25]:
all_buildings = pd.concat(bounds_etcs)

In [30]:
assert not all_buildings.index.duplicated().any()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20,20))
all_buildings.plot(ax=ax)

In [31]:
all_buildings.to_parquet('/data/uscuni-ulce/processed_data/outputs/prague/pmo_chars.pq')